In [50]:
import pandas as pd
import time, datetime
df_data = pd.read_csv('/private/tmp/voipMetrics.csv')
df_data.head()

,burstDensity,burstDuration,gapDensity,gapDuration,discardRate,endSystemDelay,Gmin,JBnominal,lossRate,signalLevel,noiseLevel,RERL,roundTripDelay,Rfactor,MOSLQ
0,8.0,0.0,19.0,9.0,1.0,27.0,9.4,15.0,6.0,3.0,29.0,7.0,32.0,79.3,4.2
1,8.0,4.0,6.0,9.0,3.0,11.0,5.1,208.0,13.0,86.0,11.0,24.0,35.0,89.2,4.3
2,6.0,2.0,15.0,3.0,2.0,21.0,3.6,68.0,11.0,105.0,10.0,16.0,9.0,86.5,4.3
3,19.0,7.0,8.0,4.0,4.0,1.0,6.5,199.0,1.0,63.0,17.0,1.0,48.0,79.9,4.3
4,10.0,3.0,12.0,6.0,3.0,10.0,9.6,139.0,6.0,102.0,20.0,2.0,44.0,91.3,4.3


In [51]:
data_train =df_data.iloc[:int(df_data.shape[0] * 0.7), :]
data_test = df_data.iloc[int(df_data.shape[0] * 0.7):, :]
print(data_train.shape, data_test.shape)

(18900, 15) (8100, 15)


In [52]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
import time
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(data_train)

MinMaxScaler(copy=True, feature_range=(-1, 1))

In [53]:
data_train = scaler.transform(data_train)
data_test = scaler.transform(data_test)

In [54]:
from keras.layers import Input, Dense, LSTM
from keras.models import Model

from keras.layers import *
from keras.models import *
from keras.optimizers import Adam

output_dim = 1
batch_size = 256 #每轮训练模型时，样本的数量
epochs = 60 #训练60轮次
seq_len = 5
hidden_size = 128


TIME_STEPS = 5
INPUT_DIM = 15

lstm_units = 64
X_train = np.array([data_train[i : i + seq_len, :] for i in range(data_train.shape[0] - seq_len)])
y_train = np.array([data_train[i + seq_len, 0] for i in range(data_train.shape[0]- seq_len)])
X_test = np.array([data_test[i : i + seq_len, :] for i in range(data_test.shape[0]- seq_len)])
y_test = np.array([data_test[i + seq_len, 0] for i in range(data_test.shape[0] - seq_len)])

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(18895, 5, 15) (18895,) (8095, 5, 15) (8095,)


In [55]:
inputs = Input(shape=(TIME_STEPS, INPUT_DIM))
#drop1 = Dropout(0.3)(inputs)

x = Conv1D(filters = 64, kernel_size = 1, activation = 'relu')(inputs)  #, padding = 'same'
#x = Conv1D(filters=128, kernel_size=5, activation='relu')(output1)#embedded_sequences
x = MaxPooling1D(pool_size = 5)(x)
x = Dropout(0.2)(x)
print(x.shape)

(?, 1, 64)


In [56]:
#lstm_out = Bidirectional(LSTM(lstm_units, activation='relu'), name='bilstm')(x)
#lstm_out = LSTM(lstm_units,activation='relu')(x)
#print(lstm_out.shape)
hidden_units1 = LSTM(units=128, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, 
                    kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', 
                    unit_forget_bias=True, dropout=0.1, recurrent_dropout=0.1, implementation=1, return_sequences=True, 
                    name='lstm1')(x)

hidden_units2 = LSTM(units=128, activation='tanh', recurrent_activation='hard_sigmoid', use_bias=True, 
                    kernel_initializer='glorot_uniform', recurrent_initializer='orthogonal', bias_initializer='zeros', 
                    unit_forget_bias=True, dropout=0.1, recurrent_dropout=0.1, implementation=1, return_sequences=True, 
                    name='lstm2')(hidden_units1)


In [57]:
from keras import backend as K
from keras.engine.topology import Layer
import numpy as np
from keras import initializers
# Attention GRU network  未用     
class AttLayer(Layer):
    def __init__(self, **kwargs):
        self.init = initializers.get('normal')
        #self.input_spec = [InputSpec(ndim=3)]
        super(AttLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape)==128
        #self.W = self.init((input_shape[-1],1))
        self.W = self.init((input_shape[-1],))
        #self.input_spec = [InputSpec(shape=input_shape)]
        self.trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)  # be sure you call this somewhere!

    def call(self, x, mask=None):
        eij = K.tanh(K.dot(x, self.W))
        
        ai = K.exp(eij)
        weights = ai/K.sum(ai, axis=1).dimshuffle(0,'x')
        
        weighted_input = x*weights.dimshuffle(0,1,'x')
        return weighted_input.sum(axis=1)

    def get_output_shape_for(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [58]:
predictions = TimeDistributed(Dense(1, activation='sigmoid'), name='predictions')(hidden_units2) 
model = Model(input=inputs, output=predictions)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 5, 15)             0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 5, 64)             1024      
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1, 64)             0         
_________________________________________________________________
lstm1 (LSTM)                 (None, 1, 128)            98816     
_________________________________________________________________
lstm2 (LSTM)                 (None, 1, 128)            131584    
_________________________________________________________________
predictions (TimeDistributed (None, 1, 1)              129       
Total para

/Users/bowchen/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`
  


In [59]:
from keras.layers import Input, Dense, merge
from keras import layers
# ATTENTION PART STARTS HERE
attention_probs = Dense(128, activation='sigmoid', name='attention_vec')(hidden_units2)
#attention_mul=layers.merge([stm_out,attention_probs], output_shape],mode='concat',concat_axis=1))
attention_mul =Multiply()([hidden_units2, attention_probs])
#attention_mul = merge([lstm_out, attention_probs],output_shape=32, name='attention_mul', mode='mul')

In [60]:
output = Dense(1, activation='sigmoid')(attention_mul)
#output = Dense(10, activation='sigmoid')(drop2)

model = Model(inputs=inputs, outputs=output)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 5, 15)        0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 5, 64)        1024        input_4[0][0]                    
__________________________________________________________________________________________________
max_pooling1d_4 (MaxPooling1D)  (None, 1, 64)        0           conv1d_4[0][0]                   
__________________________________________________________________________________________________
dropout_4 (Dropout)             (None, 1, 64)        0           max_pooling1d_4[0][0]            
__________________________________________________________________________________________________
lstm1 (LST

In [47]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, shuffle=False)
y_pred = model.predict(X_test)
print('MSE Train loss:', model.evaluate(X_train, y_train, batch_size=batch_size))
print('MSE Test loss:', model.evaluate(X_test, y_test, batch_size=batch_size))
plt.plot(y_test, label='test')
plt.plot(y_pred, label='pred')
plt.legend()
plt.show()

ValueError: Error when checking target: expected dense_9 to have 3 dimensions, but got array with shape (18895, 1)

In [3]:
print("Epoch 1/60 \
37312/37312 [==============================] - 3s 92us/step - loss: 0.1865 \
Epoch 2/60 \
37312/37312 [==============================] - 2s 46us/step - loss: 0.0514 \
Epoch 3/60 \
37312/37312 [==============================] - 2s 42us/step - loss: 0.0442 \
Epoch 4/60 \
37312/37312 [==============================] - 2s 44us/step - loss: 0.0439 \
Epoch 5/60 \
37312/37312 [==============================] - 1s 39us/step - loss: 0.0436 \
Epoch 6/60 \
37312/37312 [==============================] - 1s 35us/step - loss: 0.0432 \
Epoch 7/60 \
37312/37312 [==============================] - 1s 35us/step - loss: 0.0429 \
Epoch 8/60 \
37312/37312 [==============================] - 1s 39us/step - loss: 0.0426 \
Epoch 9/60 \
37312/37312 [==============================] - 1s 37us/step - loss: 0.0424 \
Epoch 10/60 \
37312/37312 [==============================] - 1s 34us/step - loss: 0.0422 \
Epoch 11/60 \
37312/37312 [==============================] - 1s 36us/step - loss: 0.0420 \
Epoch 12/60 \
37312/37312 [==============================] - 1s 40us/step - loss: 0.0419 \
Epoch 13/60 \
37312/37312 [==============================] - 1s 40us/step - loss: 0.0418 \
Epoch 14/60 \
37312/37312 [==============================] - 1s 38us/step - loss: 0.0417 \
Epoch 15/60 \
37312/37312 [==============================] - 2s 42us/step - loss: 0.0417 \
Epoch 16/60 \
37312/37312 [==============================] - 2s 45us/step - loss: 0.0416 \
Epoch 17/60 \
37312/37312 [==============================] - 1s 39us/step - loss: 0.0416 \
Epoch 18/60 \
37312/37312 [==============================] - 2s 42us/step - loss: 0.0416 \
Epoch 19/60 \
37312/37312 [==============================] - 2s 44us/step - loss: 0.0416 \
Epoch 20/60 \
37312/37312 [==============================] - 1s 35us/step - loss: 0.0416 \
Epoch 21/60 \
37312/37312 [==============================] - 1s 40us/step - loss: 0.0416 \
Epoch 22/60 \
37312/37312 [==============================] - 2s 41us/step - loss: 0.0416 \
Epoch 23/60 \
37312/37312 [==============================] - 1s 37us/step - loss: 0.0415 \
Epoch 24/60 \
37312/37312 [==============================] - 1s 35us/step - loss: 0.0416 \
Epoch 25/60 \
37312/37312 [==============================] - 1s 40us/step - loss: 0.0415 \
Epoch 26/60 \
37312/37312 [==============================] - 2s 41us/step - loss: 0.0415 \
Epoch 27/60 \
37312/37312 [==============================] - 2s 46us/step - loss: 0.0415 \
Epoch 28/60 \
37312/37312 [==============================] - 2s 47us/step - loss: 0.0415 \
Epoch 29/60 \
37312/37312 [==============================] - 2s 43us/step - loss: 0.0414 \
Epoch 30/60 \
37312/37312 [==============================] - 1s 39us/step - loss: 0.0414 \
Epoch 31/60 \
37312/37312 [==============================] - 2s 41us/step - loss: 0.0414 \
Epoch 32/60 \
37312/37312 [==============================] - 2s 42us/step - loss: 0.0414 \
Epoch 33/60 \
37312/37312 [==============================] - 1s 37us/step - loss: 0.0414 \
Epoch 34/60 \
37312/37312 [==============================] - 2s 44us/step - loss: 0.0414 \
Epoch 35/60 \
37312/37312 [==============================] - 2s 49us/step - loss: 0.0413 \
Epoch 36/60 \
37312/37312 [==============================] - 1s 40us/step - loss: 0.0413 \
Epoch 37/60 \
37312/37312 [==============================] - 1s 35us/step - loss: 0.0413 \
Epoch 38/60 \
37312/37312 [==============================] - 2s 48us/step - loss: 0.0413 \
Epoch 39/60 \
37312/37312 [==============================] - 1s 40us/step - loss: 0.0412 \
Epoch 40/60 \
37312/37312 [==============================] - 1s 38us/step - loss: 0.0413 \
Epoch 41/60 \
37312/37312 [==============================] - 2s 42us/step - loss: 0.0412 \
Epoch 42/60 \
37312/37312 [==============================] - 2s 41us/step - loss: 0.0412 \
Epoch 43/60 \
37312/37312 [==============================] - 1s 36us/step - loss: 0.0412 \
Epoch 44/60 \
37312/37312 [==============================] - 1s 40us/step - loss: 0.0412 \
Epoch 45/60 \
37312/37312 [==============================] - 2s 43us/step - loss: 0.0412 \
Epoch 46/60 \
37312/37312 [==============================] - 1s 37us/step - loss: 0.0412 \
Epoch 47/60 \
37312/37312 [==============================] - 1s 38us/step - loss: 0.0412 \
Epoch 48/60 \
37312/37312 [==============================] - 2s 43us/step - loss: 0.0412 \
Epoch 49/60 \
37312/37312 [==============================] - 1s 39us/step - loss: 0.0411 \
Epoch 50/60 \
37312/37312 [==============================] - 1s 37us/step - loss: 0.0411 \
Epoch 51/60 \
37312/37312 [==============================] - 2s 42us/step - loss: 0.0411 \
Epoch 52/60 \
37312/37312 [==============================] - 2s 43us/step - loss: 0.0411 \
Epoch 53/60 \
37312/37312 [==============================] - 1s 38us/step - loss: 0.0411 \
Epoch 54/60 \
37312/37312 [==============================] - 2s 47us/step - loss: 0.0410 \
Epoch 55/60 \
37312/37312 [==============================] - 2s 50us/step - loss: 0.0410 \
Epoch 56/60 \
37312/37312 [==============================] - 2s 43us/step - loss: 0.0410 \
Epoch 57/60 \
37312/37312 [==============================] - 2s 48us/step - loss: 0.0410 \
Epoch 58/60 \
37312/37312 [==============================] - 2s 50us/step - loss: 0.0410 \
Epoch 59/60 \
37312/37312 [==============================] - 2s 40us/step - loss: 0.0410 \
Epoch 60/60 \
37312/37312 [==============================] - 2s 45us/step - loss: 0.0410 \
37312/37312 [==============================] - 1s 20us/step \
MSE Train loss: 0.04148709757006819 \
15988/15988 [==============================] - 0s 15us/step \
MSE Test loss: 0.0005358342003804944")

Epoch 1/60 37312/37312 [==============================] - 3s 92us/step - loss: 0.1865 Epoch 2/60 37312/37312 [==============================] - 2s 46us/step - loss: 0.0514 Epoch 3/60 37312/37312 [==============================] - 2s 42us/step - loss: 0.0442 Epoch 4/60 37312/37312 [==============================] - 2s 44us/step - loss: 0.0439 Epoch 5/60 37312/37312 [==============================] - 1s 39us/step - loss: 0.0436 Epoch 6/60 37312/37312 [==============================] - 1s 35us/step - loss: 0.0432 Epoch 7/60 37312/37312 [==============================] - 1s 35us/step - loss: 0.0429 Epoch 8/60 37312/37312 [==============================] - 1s 39us/step - loss: 0.0426 Epoch 9/60 37312/37312 [==============================] - 1s 37us/step - loss: 0.0424 Epoch 10/60 37312/37312 [==============================] - 1s 34us/step - loss: 0.0422 Epoch 11/60 37312/37312 [==============================] - 1s 36us/step - loss: 0.0420 Epoch 12/60 37312/37312 [===========================